<a href="https://colab.research.google.com/github/Maironsantana/TCC-Sistema-de-Recomendacao-Professores/blob/main/TCC_Sistema_Recomendativo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bibliotecas

In [ ]:
!pip install unicode
import unicodedata
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from google.colab import drive

drive.mount('/content/drive')

#Carregando Dataframe

In [4]:
#Fazendo upload do arquivo csv com a seguinte estrutura:
#Orientador / Coorientador / "Tema_1" / "Tema_2" / "Tema_3" / "Tema_4" / ... (Substituindo os "temas" pelo nome do tema propriamente dito)
#Onde cada linha do csv é um trabalho com seus orientadores, coorientadores e valores de porcentagem em cada tema (Usar o script para classificar trabalhos)

TCCsDataset = pd.read_csv('/content/drive/MyDrive/.....')

In [ ]:
#Preenchendo trabalhos sem coorientadores com "-"
TCCsDataset = TCCsDataset.fillna("-")
TCCsDataset

A seguir, iremos buscar os 3 temas com maiores valores de porcentagem, assim cada trabalho terá 3 temas relacionados

In [ ]:
# Lista para armazenar os novos dados
novo_array_de_trabalhos = []

# Itera sobre as linhas do DataFrame
for _, linha in TCCsDataset.iterrows():
    # Extrai os nomes dos orientadores e coorientadores
    orientador = linha['Orientador']
    coorientador = linha['Coorientador']

    # Cria um dicionário para armazenar as pontuações dos temas
    temas_pontuacoes = {tema: linha[tema] for tema in TCCsDataset.columns[2:]}

    # Obtém os cinco temas de maior pontuação
    temas_mais_pontuados = sorted(temas_pontuacoes, key=temas_pontuacoes.get, reverse=True)[:5]

    # Adiciona os dados ao novo DataFrame
    novo_array_de_trabalhos.append({
        'Orientador': orientador,
        'Coorientador': coorientador,
        'Tema1': temas_mais_pontuados[0],
        'Tema2': temas_mais_pontuados[1],
        'Tema3': temas_mais_pontuados[2],
    })

# Cria um novo DataFrame com os dados gerados
novo_TCCsDataset = pd.DataFrame(novo_array_de_trabalhos)

# Exibe o novo DataFrame
novo_TCCsDataset


Para descobrir o tempo de experiência de um professor em um determinado tema, e feito uma relação entre Tema - Subárea da Computação - Disciplina. Sabendo o tempo de experiencia de um professor em uma determinada disciplina é possivel descobrir o mesmo para um tema da computação.

In [ ]:
#Importação de um CSV fazendo a relação de uma matéra com a subárea da computação
relacaoDisciplinaSubarea = pd.read_csv('/content/drive/MyDrive/....', index_col='MATÉRIA')
relacaoDisciplinaSubarea

In [ ]:
#Importação de um CSV fazendo a relação de um tema com a subárea da computação
relacaoTopicoSubarea = pd.read_csv('/content/drive/MyDrive/....', index_col='TÓPICO')
relacaoTopicoSubarea

In [ ]:
#Importando um CSV contendo a relação professor X MATÉRIA, onde cada célula possui o valor em horas do professor na MATÉRIA
DatasetHorasDisciplinas = pd.read_csv('/content/drive/MyDrive/....', index_col='NOME_PROFESSOR')
DatasetHorasDisciplinas

In [ ]:
#Algoritmo para calcular o tempo de experiência em horas de um professor na SUBÁREA da computação
DicionarioHorasSubareas = {}

for index, row in DatasetHorasDisciplinas.iterrows():
  for column, value in row.items():
    if index not in DicionarioHorasSubareas:
      DicionarioHorasSubareas[index] = {}
    if relacaoDisciplinaSubarea.loc[column, 'SUBÁREA'] in DicionarioHorasSubareas[index]:
      DicionarioHorasSubareas[index][relacaoDisciplinaSubarea.loc[column, 'SUBÁREA']] += value
    else:
      DicionarioHorasSubareas[index][relacaoDisciplinaSubarea.loc[column, 'SUBÁREA']] = value


#Temos agora um dataset com a relação professor X SUBÁREA
DatasetHorasSubareas = pd.DataFrame(DicionarioHorasSubareas)
DatasetHorasSubareas = DatasetHorasSubareas.transpose()
#DROPANDO MATERIAS SEM CLASSIFICAÇÃO
DatasetHorasSubareas = DatasetHorasSubareas.drop('-', axis=1)
DatasetHorasSubareas

In [ ]:
#Algoritmo para calcular o tempo de experiência em horas de um professor no TEMA da computação
DicionarioHorasTopicos = {}

for index, row in relacaoTopicoSubarea.iterrows():
  for professor, subarea in DatasetHorasSubareas.iterrows():
    if professor not in DicionarioHorasTopicos:
      DicionarioHorasTopicos[professor] = {}

    if row['SUBÁREA'] not in DatasetHorasSubareas.columns:
      DicionarioHorasTopicos[professor][index] = 0
    else:
      DicionarioHorasTopicos[professor][index] = subarea[row['SUBÁREA']]

#Temos agora um dataset com a relação professor X TEMA
DatasetHorasTopicos = pd.DataFrame(DicionarioHorasTopicos)
DatasetHorasTopicos = DatasetHorasTopicos.transpose()
DatasetHorasTopicos

#Análise dos dados

In [ ]:
# Concatenar as colunas de temas em uma única série
temas_series = pd.concat([novo_TCCsDataset['Tema1'], novo_TCCsDataset['Tema2'], novo_TCCsDataset['Tema3']])

# Calcular a contagem total de aparições para cada tema
contagem_temas = temas_series.value_counts()

# Calcular a porcentagem de aparição de cada tema
porcentagem_temas = (contagem_temas / 82)

# Ordenar os temas por porcentagem
porcentagem_temas = porcentagem_temas.sort_values(ascending=False)
df_maiores_valores = porcentagem_temas.head(10)

# Criar um gráfico de barras
plt.figure(figsize=(10, 6))
df_maiores_valores.plot(kind='bar')
plt.xlabel('TÓPICOS')
plt.ylabel('PORCENTAGEM DE TRABALHOS')
plt.show()


In [ ]:
#PARTICIPAÇÃO DOS PROFESSORES NOS TCCS APRESENTADOS
orientadores_unicos = TCCsDataset['Orientador'].append(TCCsDataset['Coorientador']).unique()

df_porcentagens_orientadores = pd.DataFrame(columns=['Porcentagem de Aparição'])

for orientador in orientadores_unicos:
    total_aparicoes = ((TCCsDataset['Orientador'] == orientador) | (TCCsDataset['Coorientador'] == orientador)).sum()

    porcentagem = total_aparicoes / len(TCCsDataset)

    df_porcentagens_orientadores.loc[orientador] = [porcentagem]

df_porcentagens_orientadores

#Pontuação individual

In [ ]:
#é feita uma soma para descobrir o valor total de participação de cada professor nos temas
PontuacaoTCC = TCCsDataset
dataset_columns = PontuacaoTCC.columns[2:]
dataset_columns

total_professores = {}

for index, row in PontuacaoTCC.iterrows():
  orientador = row['Orientador']
  coorientador = row['Coorientador']


  if orientador not in total_professores:
    total_professores[orientador] = {}
    total_professores[orientador] = row[dataset_columns]
  elif orientador in total_professores:
    total_professores[orientador] += row[dataset_columns]

  if coorientador not in total_professores:
    total_professores[coorientador] = {}
    total_professores[coorientador] = row[dataset_columns]
  elif coorientador in total_professores:
    total_professores[coorientador] += row[dataset_columns]


total_orientadores_df = pd.DataFrame(total_professores)
total_orientadores_df = total_orientadores_df.drop(columns = '-', axis=1)

total_orientadores_df

In [ ]:
#Montagem da tabela de probabilidade. Cada professor terá um valor de porcentagem calculado a partir do valor de trabalhos no tema que ele orientou, dividido pelo total de trabalhos no tema

PontuacaoTCC = total_orientadores_df

def calcular_porcentagem(linha):
    soma = linha.sum()
    if soma == 0:
        return linha  # Evita a divisão por zero, mantendo os valores originais
    return (linha / soma)

dataset_columns = PontuacaoTCC.apply(calcular_porcentagem, axis=1)

dataset_columns

In [ ]:
#Adição de valores ficticios para evitar futuramente multiplicações por 0 no algoritmo
TCCDataframe = dataset_columns.transpose()
TCCDataframe = TCCDataframe + 0.001
TCCDataframe

In [ ]:
#adição de valores ficticios para evitar multiplicações por 0 no algoritmo
DatasetHorasTopicos = DatasetHorasTopicos + 1
DatasetHorasTopicos

In [ ]:
#Calculo do valor de probabilidade do professor relacionado a quantidade de horas que ele ensinou na matéria/tema
TopicosHorasDataframe = DatasetHorasTopicos.div(DatasetHorasTopicos.sum(axis=0).replace(0,1), axis=1)

def formatar_nome_coluna(nome):
    nome = nome.lower()  # Converte para minúsculas
    nome = nome.replace(" ", "_")  # Substitui espaços por underscores
    nome = ''.join((c for c in unicodedata.normalize('NFD', nome) if unicodedata.category(c) != 'Mn'))  # Remove acentos
    return nome

TopicosHorasDataframe.columns = [formatar_nome_coluna(coluna) for coluna in TopicosHorasDataframe.columns]
TopicosHorasDataframe

In [ ]:
#É feito o merge de ambos os datasets Orientação e Horas ministradas

professores = list(set(TCCDataframe.index))
topicos = list(set(TCCDataframe.columns))

colunas = pd.MultiIndex.from_product([topicos, ['Participação', 'Horas Ministradas']])

TeachersCrossMerged = pd.DataFrame(1, index=professores, columns=colunas)

for professor in professores:
  for topico in topicos:
    if topico in TCCDataframe.columns and professor in TCCDataframe.index:
      tcc_value = TCCDataframe.at[professor, topico]
      TeachersCrossMerged.at[professor, (topico, 'Participação')] = tcc_value
    if topico in TopicosHorasDataframe.columns and professor in TopicosHorasDataframe.index:
      hours_value = TopicosHorasDataframe.at[professor, topico]
      TeachersCrossMerged.at[professor, (topico, 'Horas Ministradas')] = hours_value

TeachersCrossMerged

#Algorítimo

Primeiro modelo do algoritmo, com testes mockados

In [ ]:

#GERAR A RECOMENDAÇÃO
def gerar_recomendação(assunto1, assunto2, assunto3):
  recomendacao = {}
  for index, row in TeachersCrossMerged.iterrows():
    P_Assunto1 = TeachersCrossMerged.loc[index, (assunto1, 'Participação')]
    H_Assunto1 = TeachersCrossMerged.loc[index, (assunto1, 'Horas Ministradas')]

    P_Assunto2 = TeachersCrossMerged.loc[index, (assunto2, 'Participação')]
    H_Assunto2 = TeachersCrossMerged.loc[index, (assunto2, 'Horas Ministradas')]

    P_Assunto3 = TeachersCrossMerged.loc[index, (assunto3, 'Participação')]
    H_Assunto3 = TeachersCrossMerged.loc[index, (assunto3, 'Horas Ministradas')]


    P_total = df_porcentagens_orientadores.at[index, 'Porcentagem de Aparição']

    P = P_total * P_Assunto1 * P_Assunto2 * P_Assunto3 * H_Assunto1 * H_Assunto2 * H_Assunto3
    recomendacao[index] = P


  return recomendacao

#RESULTADO DA PREVISÃO
previsao = gerar_recomendação('aprendizado_de_maquina', 'aprendizado_de_maquina', 'aprendizado_de_maquina')


#CALCULO DA PORCENTAGEM
soma_total = sum(previsao.values())

porcentagens = {}
for autor, resultado in previsao.items():
    porcentagem = (resultado / soma_total) * 100
    porcentagens[autor] = porcentagem

porcentagens
# previsao


In [ ]:
def algoritmo_recomendacao(assunto1, assunto2, assunto3):
  previsao = gerar_recomendação(assunto1, assunto2, assunto3)
  series = pd.Series(previsao)
  qtd = 5
  melhores_professores = series.nlargest(5)
  return melhores_professores

algoritmo_recomendacao('complexidade_computacional', 'complexidade_computacional', 'complexidade_computacional')

#Validação

Testes de validação para calcular a acurácia simples

In [ ]:
#SEPARAÇÃO DOS 13 ARTIGOS PARA TESTE

contagem_orientadores = TCCsDataset['Orientador'].value_counts()

total_test_items = 13
orientador_proportions = contagem_orientadores / contagem_orientadores.sum()
artigos_por_orientador = (orientador_proportions * 20).round().astype(int)

TestTCCsDataset = pd.DataFrame()
for orientador, num_artigos in artigos_por_orientador.items():
    orientador_data = TCCsDataset[TCCsDataset['Orientador'] == orientador].sample(n=num_artigos, random_state=13)
    TestTCCsDataset = pd.concat([TestTCCsDataset, orientador_data])


#PREPARAÇÃO DO DATAFRAME DE TREINO
indices_itens_teste = TCCsDataset.index.isin(TestTCCsDataset.index)
TrainTCCsDataset = TCCsDataset[~indices_itens_teste]

#NOVO DATAFRAME DE TESTE: TestTCCsDataset
#NOVO DATAFRAME DE TREINO:
TrainTCCsDataset

A seguir, as mesmas etapas feitas anteriormente, entretando com apenas o dataframe de treino

In [ ]:
PontuacaoTrainTCC = TrainTCCsDataset
dataset_columns = PontuacaoTrainTCC.columns[2:]
dataset_columns

total_professores = {}

for index, row in PontuacaoTrainTCC.iterrows():
  orientador = row['Orientador']
  coorientador = row['Coorientador']


  if orientador not in total_professores:
    total_professores[orientador] = {}
    total_professores[orientador] = row[dataset_columns]
  elif orientador in total_professores:
    total_professores[orientador] += row[dataset_columns]

  if coorientador not in total_professores:
    total_professores[coorientador] = {}
    total_professores[coorientador] = row[dataset_columns]
  elif coorientador in total_professores:
    total_professores[coorientador] += row[dataset_columns]


train_orientadores_df = pd.DataFrame(total_professores)
train_orientadores_df = train_orientadores_df.drop(columns = '-', axis=1)

train_orientadores_df

In [ ]:
PontuacaoTrainTCC = train_orientadores_df

def calcular_porcentagem(linha):
    soma = linha.sum()
    if soma == 0:
        return linha
    return (linha / soma)

dataset_columns = PontuacaoTrainTCC.apply(calcular_porcentagem, axis=1)


TrainDataframe = dataset_columns.transpose()
TrainDataframe = TrainDataframe + 0.001
TrainDataframe

In [ ]:
professores = list(set(TrainDataframe.index))
topicos = list(set(TrainDataframe.columns))

colunas = pd.MultiIndex.from_product([topicos, ['Participação', 'Horas Ministradas']])

TeachersCrossMerged = pd.DataFrame(1, index=professores, columns=colunas)

for professor in professores:
  for topico in topicos:
    if topico in TrainDataframe.columns and professor in TrainDataframe.index:
      tcc_value = TrainDataframe.at[professor, topico]
      TeachersCrossMerged.at[professor, (topico, 'Participação')] = tcc_value
    if topico in TopicosHorasDataframe.columns and professor in TopicosHorasDataframe.index:
      hours_value = TopicosHorasDataframe.at[professor, topico]
      TeachersCrossMerged.at[professor, (topico, 'Horas Ministradas')] = hours_value

TeachersCrossMerged

In [ ]:
#GERAR A RECOMENDAÇÃO
def gerar_recomendacao1(assunto1, assunto2, assunto3):
  recomendacao = {}
  for index, row in TeachersCrossMerged.iterrows():
    P_Assunto1 = TeachersCrossMerged.loc[index, (assunto1, 'Participação')]
    H_Assunto1 = TeachersCrossMerged.loc[index, (assunto1, 'Horas Ministradas')]

    P_Assunto2 = TeachersCrossMerged.loc[index, (assunto2, 'Participação')]
    H_Assunto2 = TeachersCrossMerged.loc[index, (assunto2, 'Horas Ministradas')]

    P_Assunto3 = TeachersCrossMerged.loc[index, (assunto3, 'Participação')]
    H_Assunto3 = TeachersCrossMerged.loc[index, (assunto3, 'Horas Ministradas')]


    P_total = df_porcentagens_orientadores.at[index, 'Porcentagem de Aparição']

    P = P_total * P_Assunto1 * P_Assunto2 * P_Assunto3 * H_Assunto1 * H_Assunto2 * H_Assunto3
    recomendacao[index] = P

  return recomendacao


In [ ]:
def algoritmo_recomendacao1(assunto1, assunto2, assunto3, qtd):
  previsao = gerar_recomendacao1(assunto1, assunto2, assunto3)
  series = pd.Series(previsao)
  melhores_professores = series.nlargest(qtd)
  return melhores_professores


In [ ]:
TestDataset = TestTCCsDataset.drop('Orientador', axis=1)
TestDataset = TestDataset.drop('Coorientador', axis=1)
TestTCCsDataset

In [ ]:
def top3_cols(row):
    top3 = row.nlargest(3)
    return top3.index

mainSubject = TestDataset.apply(top3_cols, axis=1)
mainSubject

Teste de acurácia com o dataset de teste

In [ ]:
def teste_acuracia(n_professores):
  contador = 0
  for index, row in TestTCCsDataset.iterrows():
    assunto = mainSubject[index]
    assunto_1 = assunto[0]
    assunto_2 = assunto[1]
    assunto_3 = assunto[2]
    orientador = TestTCCsDataset.loc[index,'Orientador']
    coorientador = TestTCCsDataset.loc[index,'Coorientador']

    recomendacao = algoritmo_recomendacao1(assunto_1, assunto_2, assunto_3, n_professores)
    if orientador in recomendacao.index:
      contador += 1
    else:
      if coorientador in recomendacao.index:
        contador += 1

  tamanho = TestTCCsDataset.shape
  print("Levando em conta os ", n_professores," melhores: Dentre os", tamanho[0],", o algorítimo acertou", contador)

  acuracia = contador/tamanho[0]
  print("A acurácia foi de:", acuracia)

In [ ]:
teste_acuracia(1)
teste_acuracia(2)
teste_acuracia(3)
teste_acuracia(4)
teste_acuracia(5)

#CrossValidation

SEPARAÇÃO DO DATAFRAME EM 4 FOLDS

In [ ]:
TCCsDataset

In [ ]:
#PRIMEIRO FOLD
contagem_orientadores = {}
contagem_orientadores = TCCsDataset['Orientador'].value_counts()

FoldTCCsDataset1 = {}

orientador_proportions = contagem_orientadores / contagem_orientadores.sum()
artigos_por_orientador = (orientador_proportions * 41).round().astype(int)
FoldTCCsDataset1 = pd.DataFrame()

for orientador, num_artigos in artigos_por_orientador.items():
    orientador_data = TCCsDataset[TCCsDataset['Orientador'] == orientador].sample(n=num_artigos, random_state=13)
    FoldTCCsDataset1 = pd.concat([FoldTCCsDataset1, orientador_data])

#PREPARAÇÃO DO RESTANTE DO DATAFRAME DE TREINO
indices_itens_teste = TCCsDataset.index.isin(FoldTCCsDataset1.index)
SplitTCCsDataset1 = TCCsDataset[~indices_itens_teste]

FoldTCCsDataset1.shape

In [ ]:
#SEGUNDO FOLD
contagem_orientadores = {}
contagem_orientadores = SplitTCCsDataset1['Orientador'].value_counts()

FoldTCCsDataset2 = {}

orientador_proportions = contagem_orientadores / contagem_orientadores.sum()
artigos_por_orientador = (orientador_proportions * 22).round().astype(int)
FoldTCCsDataset2 = pd.DataFrame()

for orientador, num_artigos in artigos_por_orientador.items():
    orientador_data = SplitTCCsDataset1[SplitTCCsDataset1['Orientador'] == orientador].sample(n=num_artigos, random_state=13)
    FoldTCCsDataset2 = pd.concat([FoldTCCsDataset2, orientador_data])

#PREPARAÇÃO DO RESTANTE DO DATAFRAME DE TREINO
indices_itens_teste = SplitTCCsDataset1.index.isin(FoldTCCsDataset2.index)
SplitTCCsDataset2 = SplitTCCsDataset1[~indices_itens_teste]

FoldTCCsDataset2.shape

In [ ]:
#TERCEIRO FOLD
contagem_orientadores = {}
contagem_orientadores = FoldTCCsDataset1['Orientador'].value_counts()

FoldTCCsDataset3 = {}

orientador_proportions = contagem_orientadores / contagem_orientadores.sum()
artigos_por_orientador = (orientador_proportions * 21).round().astype(int)
FoldTCCsDataset3 = pd.DataFrame()

for orientador, num_artigos in artigos_por_orientador.items():
    orientador_data = FoldTCCsDataset1[FoldTCCsDataset1['Orientador'] == orientador].sample(n=num_artigos, random_state=13)
    FoldTCCsDataset3 = pd.concat([FoldTCCsDataset3, orientador_data])

#PREPARAÇÃO DO RESTANTE DO DATAFRAME DE TREINO
indices_itens_teste = FoldTCCsDataset1.index.isin(FoldTCCsDataset3.index)
SplitTCCsDataset3 = FoldTCCsDataset1[~indices_itens_teste]

FoldTCCsDataset3.shape

In [ ]:
SplitTCCsDataset2

In [ ]:
#Organizar folds
print(SplitTCCsDataset3.shape, FoldTCCsDataset2.shape, FoldTCCsDataset3.shape, SplitTCCsDataset2.shape)

In [ ]:
FoldTCCsDataset1 = SplitTCCsDataset3
FoldTCCsDataset2 = FoldTCCsDataset2
FoldTCCsDataset3 = FoldTCCsDataset3
FoldTCCsDataset4 = SplitTCCsDataset2

In [ ]:
print(FoldTCCsDataset1.shape, FoldTCCsDataset2.shape, FoldTCCsDataset3.shape, FoldTCCsDataset4.shape)

In [ ]:
def algoritmo_naive_bayes(trainDataframe, testDataframe, peso_hora, peso_orientacao):
  PontuacaoTCC = trainDataframe
  dataset_columns = PontuacaoTCC.columns[2:]
  dataset_columns

  total_professores = {}

  for index, row in PontuacaoTCC.iterrows():
    orientador = row['Orientador']
    coorientador = row['Coorientador']


    if orientador not in total_professores:
      total_professores[orientador] = {}
      total_professores[orientador] = row[dataset_columns]
    elif orientador in total_professores:
      total_professores[orientador] += row[dataset_columns]

    if coorientador not in total_professores:
      total_professores[coorientador] = {}
      total_professores[coorientador] = row[dataset_columns]
    elif coorientador in total_professores:
      total_professores[coorientador] += row[dataset_columns]


  total_orientadores_df = pd.DataFrame(total_professores)
  total_orientadores_df = total_orientadores_df.drop(columns = '-', axis=1)

  PontuacaoTCC = total_orientadores_df

  def calcular_porcentagem(linha):
      soma = linha.sum()
      if soma == 0:
          return linha
      return (linha / soma)

  dataset_columns = PontuacaoTCC.apply(calcular_porcentagem, axis=1)

  Database = dataset_columns.transpose()
  Database = Database + 0.001

  #DATASET HORAS
  professores = list(set(Database.index))
  topicos = list(set(Database.columns))

  colunas = pd.MultiIndex.from_product([topicos, ['Participação', 'Horas Ministradas']])

  TeachersCrossMerged = pd.DataFrame(1, index=professores, columns=colunas)

  for professor in professores:
    for topico in topicos:
      if topico in Database.columns and professor in Database.index:
        tcc_value = Database.at[professor, topico]
        TeachersCrossMerged.at[professor, (topico, 'Participação')] = tcc_value
      if topico in TopicosHorasDataframe.columns and professor in TopicosHorasDataframe.index:
        hours_value = TopicosHorasDataframe.at[professor, topico]
        TeachersCrossMerged.at[professor, (topico, 'Horas Ministradas')] = hours_value

  TeachersCrossMerged


  #Algoritmo
  def algoritmo(assunto1, assunto2, assunto3):
    recomendacao = {}
    for index, row in TeachersCrossMerged.iterrows():
      P_Assunto1 = TeachersCrossMerged.loc[index, (assunto1, 'Participação')] * peso_orientacao
      H_Assunto1 = TeachersCrossMerged.loc[index, (assunto1, 'Horas Ministradas')] * peso_hora

      P_Assunto2 = TeachersCrossMerged.loc[index, (assunto2, 'Participação')] * peso_orientacao
      H_Assunto2 = TeachersCrossMerged.loc[index, (assunto2, 'Horas Ministradas')] * peso_hora

      P_Assunto3 = TeachersCrossMerged.loc[index, (assunto3, 'Participação')] * peso_orientacao
      H_Assunto3 = TeachersCrossMerged.loc[index, (assunto3, 'Horas Ministradas')] * peso_hora

      P_total = df_porcentagens_orientadores.at[index, 'Porcentagem de Aparição']

      if H_Assunto1 == 0:
        P = P_total * P_Assunto1 * P_Assunto2 * P_Assunto3
        recomendacao[index] = P
      elif P_Assunto1 == 0:
        P = P_total * H_Assunto1 * H_Assunto2 * H_Assunto3
        recomendacao[index] = P
      else:
        P = P_total * P_Assunto1 * P_Assunto2 * P_Assunto3 * H_Assunto1 * H_Assunto2 * H_Assunto3
        recomendacao[index] = P

    return recomendacao

  #CHAMA A FUNÇÃO E TRAZ OS 5 PROFESSORES COM MAIOR PROBABILIDADE
  def funcao_calc(assunto1, assunto2, assunto3, qtd):
    previsao = algoritmo(assunto1, assunto2, assunto3)
    series = pd.Series(previsao)
    melhores_professores = series.nlargest(qtd)
    return melhores_professores

  TestDataset = testDataframe.drop('Orientador', axis=1)
  TestDataset = TestDataset.drop('Coorientador', axis=1)

  #TRAZ OS 3 PRINCIPAIS TEMAS DO TCC
  def top3_cols(row):
    top3 = row.nlargest(3)
    return top3.index

  mainSubject = TestDataset.apply(top3_cols, axis=1)

  #CHAMA A FUNÇÃO PARA CALCULAR A ACURÁCIA
  def teste_validacao(n_professores):
    contador = 0
    for index, row in testDataframe.iterrows():
      assunto = mainSubject[index]
      assunto_1 = assunto[0]
      assunto_2 = assunto[1]
      assunto_3 = assunto[2]
      orientador = testDataframe.loc[index,'Orientador']
      coorientador = testDataframe.loc[index,'Coorientador']

      recomendacao = funcao_calc(assunto_1, assunto_2, assunto_3, n_professores)
      if orientador in recomendacao.index:
        contador += 1
      elif coorientador in recomendacao.index:
        contador += 1
      # return (recomendacao,orientador)

    tamanho = testDataframe.shape
    print("Levando em conta os ", n_professores," melhores: Dentre os", tamanho[0],", o algorítimo acertou", contador)
    acuracia = contador/tamanho[0]
    print("A acurácia foi de:", acuracia)

  teste_validacao(1)
  teste_validacao(2)
  teste_validacao(3)
  teste_validacao(4)
  teste_validacao(5)


In [ ]:
def crossValidation(peso_hours, peso_orientation):
  print('Utilizando preso hora:', peso_hours, 'peso orientacao:', peso_orientation)

  round1TrainDataframe = {}
  round1TrainDataframe = pd.concat([FoldTCCsDataset2, FoldTCCsDataset3, FoldTCCsDataset4])
  round1TestDataframe = {}
  round1TestDataframe = FoldTCCsDataset1

  round2TrainDataframe = {}
  round2TrainDataframe = pd.concat([FoldTCCsDataset1, FoldTCCsDataset3, FoldTCCsDataset4])
  round2TestDataframe = {}
  round2TestDataframe = FoldTCCsDataset2

  round3TrainDataframe = {}
  round3TrainDataframe = pd.concat([FoldTCCsDataset1, FoldTCCsDataset2, FoldTCCsDataset4])
  round3TestDataframe = {}
  round3TestDataframe = FoldTCCsDataset3

  round4TrainDataframe = {}
  round4TrainDataframe = pd.concat([FoldTCCsDataset1, FoldTCCsDataset2, FoldTCCsDataset3])
  round4TestDataframe = {}
  round4TestDataframe = FoldTCCsDataset4

  print("Primeira Rodada:")
  algoritmo_naive_bayes(round1TrainDataframe, round1TestDataframe, peso_hours, peso_orientation)

  print("Segunda Rodada:")
  algoritmo_naive_bayes(round2TrainDataframe, round2TestDataframe, peso_hours, peso_orientation)

  print("Terceira Rodada:")
  algoritmo_naive_bayes(round3TrainDataframe, round3TestDataframe, peso_hours, peso_orientation)

  print("Quarta Rodada:")
  algoritmo_naive_bayes(round4TrainDataframe, round4TestDataframe, peso_hours, peso_orientation)


In [ ]:
crossValidation(1, 0)

In [ ]:
crossValidation(0.75, 0.25)

In [ ]:
crossValidation(0.5, 0.5)

In [ ]:
crossValidation(0.25, 0.75)

In [ ]:
crossValidation(0, 1)